In [1]:
import pandas as pd
import numpy as np

# Ejemplo de datos con 3 características y 2 clases
data = {
    'feature1': np.random.rand(100),
    'feature2': np.random.rand(100),
    'feature3': np.random.rand(100),
    'label': np.random.choice([0, 1], size=100)
}

df = pd.DataFrame(data)


In [2]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = df[['feature1', 'feature2', 'feature3']].values
y = df['label'].values

X_scaled = scaler.fit_transform(X)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization

def create_generator(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization())
    model.add(Dense(128))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization())
    model.add(Dense(3, activation='tanh'))  # Tres características generadas
    return model


In [5]:
def create_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim))
    model.add(LeakyReLU(0.2))
    model.add(Dense(64))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation='sigmoid'))  # Salida binaria: 0 o 1
    return model


In [6]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import Input

def train_gan(generator, discriminator, epochs, batch_size, X_real):
    # Configuración de optimizadores
    adam = Adam(lr=0.0002, beta_1=0.5)

    # Discriminador
    discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

    # Generador
    discriminator.trainable = False
    z = Input(shape=(100,))  # Ruido de entrada
    generated_data = generator(z)
    validity = discriminator(generated_data)
    combined = Model(z, validity)
    combined.compile(loss='binary_crossentropy', optimizer=adam)

    # Entrenamiento
    for epoch in range(epochs):
        # Entrenamiento del discriminador
        idx = np.random.randint(0, X_real.shape[0], batch_size)
        real_data = X_real[idx]
        noise = np.random.normal(0, 1, (batch_size, 100))
        generated_data = generator.predict(noise)
        
        # Entrenamiento del discriminador (real = 1, generado = 0)
        d_loss_real = discriminator.train_on_batch(real_data, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(generated_data, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Entrenamiento del generador
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))  # El generador quiere engañar al discriminador

        if epoch % 100 == 0:
            print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")


In [7]:
noise = np.random.normal(0, 1, (10, 100))  # Generar 10 muestras
generated_data = generator.predict(noise)

# Desnormalizar si es necesario
generated_data = scaler.inverse_transform(generated_data)

# Mostrar los datos generados
print(generated_data)


NameError: name 'generator' is not defined

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [7]:


# Paso 1: Preprocesar datos
# Ejemplo de datos con 3 características y 2 clases
data = {
    'feature1': np.random.rand(100),
    'feature2': np.random.rand(100),
    'feature3': np.random.rand(100),
    'label': np.random.choice([0, 1], size=100)
}

df = pd.DataFrame(data)

# Normalizar los datos de características
scaler = StandardScaler()
X = df[['feature1', 'feature2', 'feature3']].values
X_scaled = scaler.fit_transform(X)

# Paso 2: Definir el modelo generador
def create_generator(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization())
    model.add(Dense(128))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization())
    model.add(Dense(3, activation='tanh'))  # Tres características generadas
    return model

# Paso 3: Definir el modelo discriminador
def create_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim))
    model.add(LeakyReLU(0.2))
    model.add(Dense(64))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation='sigmoid'))  # Salida binaria: 0 o 1
    return model

# Paso 4: Crear el generador y el discriminador
generator = create_generator(100)  # 100 es el tamaño del vector de ruido
discriminator = create_discriminator(3)  # 3 características generadas

# Paso 5: Configurar el optimizador
adam = Adam(lr=0.0002, beta_1=0.5)

# Paso 6: Compilar el discriminador
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

# Paso 7: Crear el modelo combinado (generador + discriminador)
discriminator.trainable = False
z = Input(shape=(100,))  # Ruido de entrada
generated_data = generator(z)
validity = discriminator(generated_data)
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=adam)

# Paso 8: Generar datos
noise = np.random.normal(0, 1, (10, 100))  # Generar 10 muestras de ruido
generated_data = generator.predict(noise)

# Desnormalizar los datos generados
generated_data = scaler.inverse_transform(generated_data)

# Mostrar los datos generados
print(generated_data)


2024-12-19 18:48:21.999983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-19 18:48:22.654631: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 950 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:c3:00.0, compute capability: 8.6
/home/hpinto/.local/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


NameError: name 'Input' is not defined